# Створення музики за допомогою Magenta

[Magenta](https://magenta.tensorflow.org/) бібліотека для мови програмування Python для генерації музики та образотворчого мистецтва.

# Локальні налаштування

Завантаження бібліотек:

In [ ]:
import sys
!{sys.executable} -m pip install magenta==2.1.3 pretty-midi==0.2.9 pyFluidSynth==1.3.0 tensorflow==2.3.1

print("SUCCESS")

In [ ]:
print("Importing libraries...")

import magenta
import note_seq
import tensorflow

from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.polyphony_rnn import polyphony_sequence_generator
from magenta.models.shared import sequence_generator
from magenta.models.shared import sequence_generator_bundle

from note_seq.protobuf import generator_pb2
from note_seq.protobuf import music_pb2

print(magenta.__version__)
print(tensorflow.__version__)

print("SUCCESS")

Завантаження моделей:

In [ ]:
print("Downloading model bundle. This will take less than a minute...")

note_seq.notebook_utils.download_bundle("lookback_rnn.mag", "models/")
note_seq.notebook_utils.download_bundle("polyphony_rnn.mag", "models/")

print("SUCCESS")

Ініціалізація моделей:

In [ ]:
lookback_bundle = sequence_generator_bundle.read_bundle_file("models/lookback_rnn.mag")
lookback_generator_map = melody_rnn_sequence_generator.get_generator_map()
lookback_rnn = lookback_generator_map['lookback_rnn'](checkpoint=None, bundle=lookback_bundle)
lookback_rnn.initialize()

In [ ]:
polyphony_bundle = sequence_generator_bundle.read_bundle_file("models/polyphony_rnn.mag")
polyphony_generator_map = polyphony_sequence_generator.get_generator_map()
polyphony_rnn = polyphony_generator_map["polyphony"](checkpoint=None, bundle=polyphony_bundle)
polyphony_rnn.initialize()

# Продовження одноголосної мелодії

Імпорт одноголосного музичного файлу "Щедрик":

In [ ]:
schedrik_mono = note_seq.midi_file_to_note_sequence("schedrik_mono.mid")

note_seq.plot_sequence(schedrik_mono)
note_seq.play_sequence(schedrik_mono, synth=note_seq.fluidsynth)

Налаштування моделі:

In [ ]:
input_melody = schedrik_mono
rnn = lookback_rnn
num_steps = 512
temperature = 0.5

last_end_time = max(n.end_time for n in input_melody.notes) if input_melody.notes else 0
qpm = input_melody.tempos[0].qpm 
seconds_per_step = 60.0 / qpm / rnn.steps_per_quarter

start_time=last_end_time + seconds_per_step
end_time = num_steps * seconds_per_step

generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = temperature
generate_section = generator_options.generate_sections.add(start_time=start_time, end_time=end_time)

Власне, продовження мелодії:

In [ ]:
continued_melody = rnn.generate(input_melody, generator_options)

note_seq.plot_sequence(continued_melody)
note_seq.play_sequence(continued_melody, synth=note_seq.fluidsynth)

# Продовження багатоголосної мелодії

Імпорт багатоголосного музичного файлу "Щедрик":

In [ ]:
schedrik_poly = note_seq.midi_file_to_note_sequence("schedrik_poly.mid")

note_seq.plot_sequence(schedrik_poly)
note_seq.play_sequence(schedrik_poly, synth=note_seq.fluidsynth)

Налаштування моделі:

In [ ]:
input_melody = schedrik_poly
rnn = polyphony_rnn
num_steps = 640
temperature = 0.3

last_end_time = max(n.end_time for n in input_melody.notes) if input_melody.notes else 0
qpm = input_melody.tempos[0].qpm 
seconds_per_step = 60.0 / qpm / rnn.steps_per_quarter

start_time=last_end_time + seconds_per_step
end_time = num_steps * seconds_per_step

generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = temperature
generate_section = generator_options.generate_sections.add(start_time=start_time, end_time=end_time)

Власне, продовження мелодії:

In [ ]:
continued_melody = rnn.generate(input_melody, generator_options)

note_seq.plot_sequence(continued_melody)
note_seq.play_sequence(continued_melody, synth=note_seq.fluidsynth)